In [1]:
import torch
import numpy as np
import gymnasium as gym
from matplotlib import pyplot as plt
from IPython.display import clear_output

import sys
sys.path.append('../../..')

from DDPG import DDPGAgent

from utils.logger import Logger
from utils.trainer import Trainer

In [2]:
env_string = "Pendulum-v1"

In [3]:
env = gym.make(env_string)

In [4]:
agent = DDPGAgent(env.observation_space, env.action_space)

In [5]:
trainer = Trainer()
ep_rewards = trainer.train(env, agent, n_episodes=30000, test_every=100, noise=0.2)

Step 101/30000:
                    test reward: -1555.07
                    mean reward: -1522.63, max reward: -956.80
                    Avg. step number: 199.0, Avg ep duration: 0.059s
                    Avg. train duration: 0.521s
            


In [ ]:
def play(env_string, agent, n_episodes=10, noise=0):
    env = gym.make(env_string, render_mode='human')
    steps, rewards, observations, actions = Trainer.run(env, agent, n_episodes=n_episodes, noise=noise)
    env.close()
    return steps, rewards, observations, actions

In [ ]:
steps, rewards, observations, actions = play("Pendulum-v1", agent, n_episodes=1, noise=0)
print(np.sum(rewards))

-116.79800678190446


In [ ]:
torch.save(agent.state(), 'checkpoint_lunar.pth')

In [ ]:
agent_state = torch.load('checkpoint_lunar.pth')
agent.restore_state(agent_state)